In [16]:
#필요한 라이브러리들 불러옴
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn

from nltk.tokenize import word_tokenize

In [17]:
Adata = pd.read_csv('데이터_최종.csv')
# 데이터의 NAN값 제거
Adata2 = Adata.dropna(axis=0)

#일부 크롤링한 데이터중 \nBook 같은 잘못된 데이터가 들어 있어 제거
Adata2['title']= Adata2['title'].str.rstrip('\nBook')


Adata2.head()

,title,theme
0,Mastering QuickBooks 2020,Business&Other
1,Hands-On Microsoft Teams,Business&Other
2,Microsoft 365 and SharePoint Online Cookb,Business&Other
3,Microsoft Power Platform Enterprise Architecture,Business&Other
4,Odoo 12 Development Cookbook - Third Edition,Business&Other


In [19]:
"""
지난 프로젝트와의 비교를 위해 생성한 토큰 생성기
"""
# 트위터나 페이스북 같은 SNS 텍스트를 분석할때 사용하는 토큰함수
from nltk.tokenize.casual import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
#tokenizer = TfidfVectorizer(tokenizer=casual_tokenize)
#tokens = tokenizer.fit(Adata['title'])

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TreebankWordTokenizer
vect = CountVectorizer(tokenizer=casual_tokenize,min_df=3, ngram_range=(1,2))

In [21]:
tokens = vect.fit(Adata['title'])

C:\Users\Chicken is Right\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [13]:
#tokens.vocabulary_
#지난번 프로젝트에서 생성하던 토큰 방식임
tokens.get_feature_names()

['!',
 '! (',
 '! )',
 '! ...',
 '! :',
 '! all',
 '! book',
 '! for',
 '! with',
 '"',
 '" (',
 '" )',
 '" 6x9',
 '" auth',
 '" inches',
 '" x',
 '" x8',
 '#',
 '# (',
 '# )',
 '# ,',
 '# .',
 '# 1',
 '# 2005',
 '# 2008',
 '# 2010',
 '# 2012',
 '# 3.0',
 '# 4.0',
 '# 5.0',
 '# 6',
 '# 6.0',
 '# 7',
 '# 7.0',
 '# 7.1',
 '# 8',
 '# 8.0',
 '# :',
 '# and',
 '# book',
 '# by',
 '# challenge',
 '# cookbook',
 '# data',
 '# database',
 '# developers',
 '# for',
 '# functional',
 '# in',
 '# language',
 '# machine',
 '# multithreaded',
 '# programmer',
 '# programming',
 '# skills',
 '# step',
 '# with',
 '$',
 '%',
 '&',
 '& 10',
 '& 11',
 '& 17',
 '& 1z0',
 '& 2',
 '& 20',
 '& 2016',
 '& 220-1002',
 '& 220-702',
 '& 220-802',
 '& 220-902',
 '& a',
 '& a30',
 '& access',
 '& accounting',
 '& administration',
 '& alexa',
 '& algorithms',
 '& analytics',
 '& answers',
 '& applications',
 '& ass',
 '& assurance',
 '& bartlett',
 '& beyond',
 '& big',
 '& black',
 '& computer',
 '& computer-vis

In [22]:
"""
워드 벡터에 어휘를 추가하기 위한 코드
아마존 데이터 추가

"""

from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
tokens = []
for token in Adata['title']:
    tokens.append(tokenizer.tokenize(token))


In [31]:
#워드벡터에 Orelly 데이터 추가
Odata = pd.read_excel('Oreilly_IT_Book.xlsx')
for token in Odata['title']:
    tokens.append(tokenizer.tokenize(token))
    
for token in Odata['description']:
    tokens.append(tokenizer.tokenize(token))

In [47]:
#워드 벡터에 Packt의 Description 추가
Pdata = pd.read_excel('Packt.xlsx')
for token in Pdata['title']:
    tokens.append(tokenizer.tokenize(token))

In [103]:
#워드 벡터에 추가적으로 크롤링한 Amazon 데이터 추가
Amdata = pd.read_excel('Amazon.xlsx')
for token in Odata['title']:
    tokens.append(tokenizer.tokenize(token))
    
for token in Odata['description']:
    tokens.append(tokenizer.tokenize(token))

In [24]:
from gensim.models.word2vec import Word2Vec

In [48]:
#워드 벡터 생성

num_features = 300 # 단어 벡터 차원수
min_word_count = 3 # 지난 토큰 생성기와 동일하게 min_df =3 으로 입력
window_size=5   # 워드 벡터의 문맥을 파악하는 길이 (특정 단어가 들어갈 위치의 양 옆으로 window_size만큼의 단어들로 들어갈 단어 예측)
subsampling =1e-4  #많은 양의 토큰이 있을경우 처리 (빈도수가 많이 나오는 단어는 학습량을 의도적으로 줄일때 사용하는 매개변수)

word_vector = Word2Vec(tokens, size=num_features, min_count=min_word_count,window=window_size,sample=subsampling)

In [104]:
word_vector.init_sims(replace=True)

In [105]:
#생성한 워드 벡터 저장하는 코드

model_name = "p15_word2vec_3"
word_vector.save(model_name)

In [ ]:
#p15_word2vec_2 19:29 Orelly data 추가한 버전

In [73]:
#모델 불러올때
model_name = "p15_word2vec_3"
word_vector = Word2Vec.load(model_name)
word_vector.most_similar('data')

C:\Users\Chicken is Right\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[('learning', 0.8743589520454407),
 ('web', 0.8708198070526123),
 ('machine', 0.8542468547821045),
 ('applications', 0.8517473936080933),
 ('analysis', 0.8435206413269043),
 ('problems', 0.8325647115707397),
 ('modern', 0.8236908912658691),
 ('algorithms', 0.8201415538787842),
 ('models', 0.8180030584335327),
 ('development', 0.8131026029586792)]

In [9]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D

Using TensorFlow backend.
C:\Users\Chicken is Right\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Chicken is Right\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Chicken is Right\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Chicken is Right\Anaconda3\lib\site-pa

In [3]:
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample)
        sample_vecs = []
        """
        it 도서 분류를 위해 만든 word2벡터를 불러옴
        워드 벡터에 없는 단어는 분류에 필요없는 토큰으로 여기고 제거
        """
        for token in tokens:
            try:
                sample_vecs.append(word_vector[token])
                
            except KeyError:
                pass
            
        vectorized_data.append(sample_vecs)
    return vectorized_data

In [4]:
#각 훈련용 데이터의 단어 길이를 맞추기 위한 함수
def pad(data, maxlen):
    new_data = []
    
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
        
    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
        
    return new_data

In [26]:
# 지난번 프로젝트와 동일한 데이터를 훈련용과 테스트용으로 나눔

from sklearn.model_selection import train_test_split
X = tokenize_and_vectorize(Adata['title'])
y = Adata['theme']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100,test_size=0.15)

C:\Users\Chicken is Right\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [229]:
#훈련 데이터를 CNN모델에 학습 시킬 자료를 3차원 벡터로 변환
X_train = pad(X_train, maxlen)
X_train = np.reshape(X_train, (len(X_train), maxlen, embedding_dims))

X_test = pad(X_test, maxlen)
X_test = np.reshape(X_test, (len(X_test), maxlen, embedding_dims))

In [45]:
label_list = y_train.unique() # 라벨로 사용할 9개의 값 따로 저장
label_list

array(['IoT&Hardware', 'Security', 'Cloud&Networking', 'Data',
       'Business&Other', 'Programming', 'WebDevelopment', 'Mobile',
       'Game Development'], dtype=object)

In [5]:
#영문으로 된 라벨을 훈련용으로 변환하기 위해 숫자 0~8로 바꾸는 함수


def labeling(data):
    y_train_shape = []
    
    for label in data:
        for i in range(0,9):
            if label_list[i] == label :
                y_train_shape.append(i)
                
    return y_train_shape

In [230]:
#영문으로 된 라벨을 숫자(0~8)로 바꿈

y_train = labeling(y_train)
y_test = labeling(y_test)

In [240]:
#라벨 데이터를 원핫 인코딩을 통해 훈련용으로 변환
from tensorflow.keras.utils import to_categorical
y_train = np.array(y_train)
y_train_cat = to_categorical(y_train, 9)

y_test = np.array(y_test)
y_test_cat = to_categorical(y_test, 9)

In [241]:
y_train_cat.shape

(15908, 9)

In [32]:
maxlen = 10 # 각 훈련데이터당 토큰의 최대 개수
batch_size = 20 # 역전파 하기까지 처리해야하는 데이터수
embedding_dims = 300  #토큰 벡터의 길이(차원수)
filters = 250    #훈련에 사용할 필터 개수
kernel_size = 2 #1차원 필터의 너비
hidden_dims = 250 #은닉층 개수
epochs = 12 #훈련 반복 횟수

In [234]:
#활성화 함수는 relu 사용, 토큰 벡터의 길이는 워드 벡터와 마찬가지로 300으로 맞춤
# feature map의 남는 공간을 채울 수 있도록 padding 활성화, stride는 띄어넘는 단어가 없도록 1로 설정
model = Sequential()

model.add(Conv1D(
filters, kernel_size, padding='valid', activation='relu', strides=1, input_shape=(maxlen, embedding_dims)))

In [235]:
# pooling 활성화
model.add(GlobalMaxPooling1D())

In [236]:
#훈련 과정중 과대적합을 피하기 위해 드롭아웃을 사용
#20% 데이터가 훈련 과정중 무작위로 폐기됨

model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

In [237]:
#출력층은 라벨 수에 맞쳐 9개로 입력
model.add(Dense(9))
model.add(Activation('sigmoid'))

In [238]:
#손실함수로 분류명이 여러개일때 사용하는 categorical_crossentropy 사용하여 컴파일
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [242]:
"""
이번 프로젝트에서는 책 제목 특성상 카테고리가 겹치는 경우가 많으므로 테스트 셋의 예측결과를 평가지표로 사용하지 않고
최고 정확도와 유사한 예측값들은 함께 다중라벨링 처리를 할 수 있는지를 모델의 성능 평가 지표로 활용함

"""

model.fit(X_train, y_train_cat, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test_cat))

Train on 15908 samples, validate on 2808 samples
Epoch 1/12
15908/15908 [==============================] - 20s 1ms/step - loss: 1.6561 - accuracy: 0.3597 - val_loss: 1.4474 - val_accuracy: 0.4580
Epoch 2/12
15908/15908 [==============================] - 19s 1ms/step - loss: 1.3681 - accuracy: 0.5051 - val_loss: 1.3216 - val_accuracy: 0.5078
Epoch 3/12
15908/15908 [==============================] - 20s 1ms/step - loss: 1.2527 - accuracy: 0.5519 - val_loss: 1.3179 - val_accuracy: 0.5377
Epoch 4/12
15908/15908 [==============================] - 19s 1ms/step - loss: 1.1731 - accuracy: 0.5813 - val_loss: 1.2507 - val_accuracy: 0.5566
Epoch 5/12
15908/15908 [==============================] - 20s 1ms/step - loss: 1.1166 - accuracy: 0.6032 - val_loss: 1.2063 - val_accuracy: 0.5673
Epoch 6/12
15908/15908 [==============================] - 26s 2ms/step - loss: 1.0636 - accuracy: 0.6143 - val_loss: 1.1948 - val_accuracy: 0.5826
Epoch 7/12
15908/15908 [==============================] - 22s 1ms/ste

In [262]:
#만든 모델 저장
model.save('nlp_model.h5')

In [ ]:
""""""""""""""

In [74]:
#만든 모델 불러오기
from keras.models import load_model
model2 = load_model('nlp_model.h5')

In [75]:
# 훈련에 사용하지 않은 베스트 셀러 데이터 불러옴
pro = pd.read_csv('prog_book.csv')

In [76]:
pro.head()

,Rating,Reviews,Book_title,Description,Number_Of_Pages,Type,Price
0,4.17,"3,829",The Elements of Style,This style manual offers practical advice on i...,105,Hardcover,9.323529
1,4.01,"1,406","The Information: A History, a Theory, a Flood","James Gleick, the author of the best sellers C...",527,Hardcover,11.000000
2,3.33,0,Responsive Web Design Overview For Beginners,In Responsive Web Design Overview For Beginner...,50,Kindle Edition,11.267647
3,3.97,"1,658",Ghost in the Wires: My Adventures as the World...,If they were a hall of fame or shame for compu...,393,Hardcover,12.873529
4,4.06,"1,325",How Google Works,Both Eric Schmidt and Jonathan Rosenberg came ...,305,Kindle Edition,13.164706


In [77]:
test = tokenize_and_vectorize(pro['Book_title'])

C:\Users\Chicken is Right\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [78]:
test_train = pad(test, maxlen)
test_train = np.reshape(test_train, (len(test_train), maxlen, embedding_dims))

In [79]:
pre =model2.predict(test_train)

In [80]:
#최고치랑 비슷한 예측값들은 다중 라벨링 처리
def multi_label(data):
    count = []
    for i in range(0, len(data)):
        best = 0
        co = []

        for j in range(0,9):
        
            if pre[i][j]> best :
                best = data[i][j]
        for j in range(0,9):
        
            if pre[i][j] > best*0.9:
                co.append(j)
        count.append(co)
    return count

In [81]:
#숫자로 된 라벨을 다시 영문으로 바꾸는 함수


def labeling_c(data):
    str_label = []
    for label in data:
        sub_str = []
        print(label)
        for i in range(0,len(label)):
            for j in range(0,9):
                
                if j == label[i] :
                    sub_str.append(label_list[j])
        str_label.append(sub_str)      
    return str_label

In [82]:
labeling_c(multi_label(pre))

[3]
[4]
[3]
[2]
[1]
[3]
[1]
[4]
[1]
[3, 6]
[4]
[2]
[4]
[4]
[2]
[7]
[2, 4]
[1]
[4]
[2]
[0]
[1]
[4]
[7]
[8]
[4]
[7]
[1]
[4]
[7]
[6]
[3]
[7]
[4]
[4]
[7]
[4]
[4]
[4, 7]
[4]
[4]
[3]
[3]
[4]
[3]
[4]
[4]
[4]
[1]
[7]
[3]
[3]
[7]
[4]
[4, 6]
[3, 4]
[3]
[3]
[3]
[4]
[3]
[4]
[3]
[4]
[3]
[7]
[2]
[4]
[4]
[3]
[4]
[3]
[4]
[4]
[1, 6]
[3]
[8]
[1]
[2]
[1]
[4]
[8]
[4]
[1]
[7]
[7]
[3]
[6]
[6]
[8]
[4]
[4]
[3]
[2]
[6]
[3]
[3]
[3]
[2]
[6]
[3]
[3]
[3]
[4]
[4]
[3]
[3]
[4]
[4]
[3]
[4]
[3]
[3]
[4]
[3]
[3]
[4]
[3]
[3]
[4]
[3]
[1]
[4]
[1]
[3]
[3]
[3]
[6]
[1]
[1]
[6]
[4]
[3]
[6]
[4]
[1]
[1]
[4]
[4]
[3, 4]
[4]
[3]
[6]
[3]
[3]
[4]
[3]
[3]
[8]
[2]
[4]
[4]
[3]
[4]
[3]
[4]
[3]
[3]
[3]
[4]
[3, 4]
[3]
[4]
[4]
[0]
[3]
[3]
[4]
[3]
[3]
[3]
[4]
[3]
[3]
[4]
[4]
[8]
[8]
[3, 4]
[3]
[4]
[3]
[8]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[4]
[3]
[6]
[3]
[3]
[4]
[3]
[4]
[3]
[4]
[3]
[3]
[6]
[3]
[3]
[3]
[3]
[8]
[3]
[4]
[4]
[4]
[4]
[4]
[8]
[4]
[1, 4]
[4, 6]
[3]
[3]
[2, 4]
[3]
[1, 3]
[4]
[4]
[3]
[3]
[3]
[3]
[3]
[4]
[4]
[4]
[4]
[4]
[8]
[4]
[8]
[8]
[3, 8

[['Data'],
 ['Business&Other'],
 ['Data'],
 ['Cloud&Networking'],
 ['Security'],
 ['Data'],
 ['Security'],
 ['Business&Other'],
 ['Security'],
 ['Data', 'WebDevelopment'],
 ['Business&Other'],
 ['Cloud&Networking'],
 ['Business&Other'],
 ['Business&Other'],
 ['Cloud&Networking'],
 ['Mobile'],
 ['Cloud&Networking', 'Business&Other'],
 ['Security'],
 ['Business&Other'],
 ['Cloud&Networking'],
 ['IoT&Hardware'],
 ['Security'],
 ['Business&Other'],
 ['Mobile'],
 ['Game Development'],
 ['Business&Other'],
 ['Mobile'],
 ['Security'],
 ['Business&Other'],
 ['Mobile'],
 ['WebDevelopment'],
 ['Data'],
 ['Mobile'],
 ['Business&Other'],
 ['Business&Other'],
 ['Mobile'],
 ['Business&Other'],
 ['Business&Other'],
 ['Business&Other', 'Mobile'],
 ['Business&Other'],
 ['Business&Other'],
 ['Data'],
 ['Data'],
 ['Business&Other'],
 ['Data'],
 ['Business&Other'],
 ['Business&Other'],
 ['Business&Other'],
 ['Security'],
 ['Mobile'],
 ['Data'],
 ['Data'],
 ['Mobile'],
 ['Business&Other'],
 ['Business&Othe

In [246]:
df = pd.DataFrame()

In [247]:
df['title'] = pro['Book_title']

In [248]:
df['prediction'] = labeling_c(count)

[6]
[7]
[2, 3]
[7]
[2]
[7]
[2]
[4]
[2]
[8]
[4]
[7]
[2, 6]
[4]
[2, 7]
[2, 7]
[4]
[6]
[7]
[7]
[6]
[1]
[6]
[7]
[8]
[1]
[2]
[7]
[4]
[7]
[8]
[3]
[7]
[4]
[6]
[7]
[1]
[4]
[1, 7]
[1]
[1]
[3]
[1]
[6]
[3]
[6]
[3]
[2]
[3]
[1]
[3]
[3]
[7]
[7]
[6]
[4]
[3]
[8]
[3]
[4]
[3]
[1]
[6]
[4]
[3]
[2]
[6]
[7]
[4]
[6]
[4]
[3]
[4]
[4, 7]
[2, 4]
[3]
[8]
[1]
[1]
[1]
[3]
[8]
[6]
[1]
[2, 3]
[7]
[3]
[2]
[2]
[3]
[4]
[4]
[3]
[2]
[1]
[1]
[3]
[7]
[2]
[1]
[3]
[3]
[3]
[4]
[6]
[3]
[3]
[3]
[4]
[1]
[4]
[3]
[3]
[3]
[3]
[3]
[7]
[3]
[1]
[4]
[3]
[1]
[4]
[6]
[3]
[3]
[3]
[6]
[1]
[1, 5]
[6]
[7]
[4]
[6]
[3]
[1]
[1]
[7]
[3]
[4]
[3]
[6]
[1]
[3]
[1]
[1]
[1]
[3]
[8]
[1]
[7]
[4]
[3]
[2]
[3]
[7]
[3]
[3]
[3]
[4]
[3]
[4]
[4]
[1, 3]
[0]
[3]
[3]
[4]
[3]
[3]
[3]
[1, 3, 6]
[1]
[3]
[3]
[3]
[8]
[8]
[1]
[3]
[6]
[3]
[8]
[3]
[3]
[3]
[3]
[3]
[3]
[4]
[2]
[3]
[3]
[2, 3]
[3]
[7]
[3, 4]
[1, 3]
[3]
[1]
[3]
[3]
[4]
[3]
[3]
[3]
[3]
[8]
[4]
[4]
[1]
[4]
[1]
[6]
[8]
[4]
[1]
[6]
[3]
[3]
[1]
[3]
[1]
[1]
[4]
[3]
[7]
[3]
[1]
[3]
[4]
[1]
[1]
[2]
[4]
[6]
[1]
[8]
[4]

In [249]:
df.to_excel('결과_2141.xlsx')

In [83]:
language = pd.read_csv('language.csv')

In [84]:
lan_test = tokenize_and_vectorize(language['language'])

C:\Users\Chicken is Right\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [85]:
lan_prediction = pad(lan_test, maxlen)
lan_prediction = np.reshape(lan_prediction, (len(lan_prediction), maxlen, embedding_dims))

In [86]:
pre_lan =model2.predict(lan_prediction )

In [267]:
df_lan = pd.DataFrame()

In [268]:
df_lan['language'] = language['language']

In [269]:
"""
word2vec에서 아직 프로그래밍 언어의 학습이 부족함
아직 워드 투 벡터 구축에 사용된 데이터의 양이 부족해 발생한 문제로 추정되어
데이터의 양을 늘려야될 것 같음

"""

df_lan['predictiobn'] = labeling_c(multi_label(pre_lan))

[]
[1, 2, 3, 4, 6, 7]
[]
[]
[1, 2, 3, 4, 6, 7]
[3]
[]
[]
[1]
[3]
[2, 4, 7]
[]
[4]
[1, 2, 3, 4, 7]
[1, 2, 3, 4, 6, 7]
[2, 7]
[2]
[1, 6]
[]


In [271]:
df_lan.to_excel('언어와 관련된 기술.xlsx')

In [13]:
result = pd.read_excel('결과_2141.xlsx')

In [15]:
"""
딥러닝 프로젝트 평가:
지난 프로젝트로 만든 단어 빈도수에 의존하는 자연어 처리 모델에 비해 다중 라벨링이나 
책의 제목을 카테고리에 맞게 분류하는 성능이 향상되었음을 확인할 수 있었음
아직 분류 성능이 만족스러운 정도는 아니지만 프로젝트의 목표인 딥러닝을 통해 성능 개선이 이루어진다는 것을 확인해볼 수 있었음 
앞으로 매개변수의 최적화나 word2vec의 어휘량 보강등 프로젝트 시간상 끝내지 못한 모델 최적화를 마무리하면
모델 성능을 더욱 끌어올릴 수 있을 것이라고 기대됨

"""


result['prediction'].value_counts()

['Programming']                              84
['IoT&Hardware']                             46
['Data']                                     43
['Business&Other']                           25
['Security']                                 23
['Game Development']                         19
['Cloud&Networking']                         14
['Cloud&Networking', 'Programming']           3
['Cloud&Networking', 'Security']              2
['Data', 'Programming']                       2
['Data', 'Security']                          1
['Data', 'WebDevelopment']                    1
['IoT&Hardware', 'Security']                  1
['Mobile']                                    1
['Data', 'Programming', 'Business&Other']     1
['Programming', 'IoT&Hardware']               1
['Data', 'Cloud&Networking']                  1
['Data', 'Business&Other']                    1
['Cloud&Networking', 'Business&Other']        1
['Cloud&Networking', 'IoT&Hardware']          1
Name: prediction, dtype: int64